DATA PREPERATION

In [5]:
import pandas as pd

# Load the CSV files
matches = pd.read_csv(r"D:\VELAI THEDUM PADALAM\IPL\DATASET\matches.csv")
players = pd.read_csv(r"D:\VELAI THEDUM PADALAM\IPL\DATASET\players.csv")
deliveries = pd.read_csv(r"D:\VELAI THEDUM PADALAM\IPL\DATASET\deliveries.csv")

# Merge players with matches to get team1/team2 context
matches.rename(columns={"id": "match_id"}, inplace=True)
players_with_match = players.merge(matches[['match_id', 'team1', 'team2', 'winner']], on='match_id', how='left')

# Separate players into team1 and team2
team1_players = players_with_match[players_with_match['team'] == players_with_match['team1']]
team2_players = players_with_match[players_with_match['team'] == players_with_match['team2']]

# Group player lineups
team1_grouped = team1_players.groupby('match_id')['playing_xi'].apply(list).reset_index(name='team1_players')
team2_grouped = team2_players.groupby('match_id')['playing_xi'].apply(list).reset_index(name='team2_players')

# Merge player lists into match data
match_data = matches.merge(team1_grouped, on='match_id', how='left')
match_data = match_data.merge(team2_grouped, on='match_id', how='left')

# Create binary label for team1 win
match_data['team1_win'] = (match_data['winner'] == match_data['team1']).astype(int)

# Save match + team player data
match_data.to_csv("processed_match_data.csv", index=False)

# -------------------------------------
# Player performance summary from deliveries
# -------------------------------------

# Batting stats
batting = deliveries.groupby('batter').agg(
    total_runs=('batsman_runs', 'sum'),
    balls_faced=('ball', 'count')
).reset_index()
batting['batting_avg'] = batting['total_runs'] / batting['balls_faced']
batting['strike_rate'] = batting['total_runs'] / batting['balls_faced'] * 100

# Bowling stats (only for dismissals)
dismissals = ['bowled', 'caught', 'lbw', 'stumped', 'caught and bowled', 'hit wicket']
bowling = deliveries[deliveries['dismissal_kind'].isin(dismissals)]
wickets = bowling.groupby('bowler').size().reset_index(name='wickets')

# Merge batting and bowling into player performance
player_perf = pd.merge(batting, wickets, left_on='batter', right_on='bowler', how='outer')

# Save player performance summary
player_perf.to_csv("player_performance_summary.csv", index=False)

print("✅ Done: Created 'processed_match_data.csv' and 'player_performance_summary.csv'")


✅ Done: Created 'processed_match_data.csv' and 'player_performance_summary.csv'


In [15]:
import pandas as pd
import numpy as np

# Load dataset
deliveries_df = pd.read_csv(r"D:\VELAI THEDUM PADALAM\IPL\DATASET\deliveries.csv")

# Batting stats
batting_stats = deliveries_df.groupby(['match_id', 'batter']).agg(
    total_runs=('batsman_runs', 'sum'),
    balls_faced=('ball', 'count'),
    num_4s=('batsman_runs', lambda x: (x == 4).sum()),
    num_6s=('batsman_runs', lambda x: (x == 6).sum())
).reset_index()

# Calculate strike rate
batting_stats['strike_rate'] = batting_stats['total_runs'] / batting_stats['balls_faced'] * 100

# Dismissals per batter per match (for batting average)
dismissals = deliveries_df[deliveries_df['player_dismissed'].notnull()]
dismissals_count = dismissals.groupby(['match_id', 'player_dismissed']).size().reset_index(name='dismissals')

# Merge dismissals into batting stats
batting_stats = batting_stats.merge(dismissals_count, left_on=['match_id', 'batter'], right_on=['match_id', 'player_dismissed'], how='left')
batting_stats['dismissals'] = batting_stats['dismissals'].fillna(0)

# Batting average (only if dismissed, else NaN or total_runs for undismissed)
batting_stats['batting_avg'] = batting_stats.apply(
    lambda row: row['total_runs'] / row['dismissals'] if row['dismissals'] > 0 else np.nan, axis=1
)

# Add 50s and 100s
batting_stats['fifties'] = batting_stats['total_runs'].apply(lambda x: 1 if 50 <= x < 100 else 0)
batting_stats['centuries'] = batting_stats['total_runs'].apply(lambda x: 1 if x >= 100 else 0)

# Bowling stats
valid_dismissals = ['bowled', 'caught', 'lbw', 'stumped', 'caught and bowled', 'hit wicket']
bowling_df = deliveries_df.copy()
bowling_df['is_legal_delivery'] = ~bowling_df['extras_type'].isin(['wide', 'no ball'])
bowling_df['legal_ball'] = bowling_df['is_legal_delivery'].astype(int)

# Group bowling stats
bowling_stats = bowling_df.groupby(['match_id', 'bowler']).agg(
    legal_deliveries=('legal_ball', 'sum'),
    wickets=('player_dismissed', lambda x: x.notnull().sum())
).reset_index()

bowling_stats['overs'] = bowling_stats['legal_deliveries'] // 6 + (bowling_stats['legal_deliveries'] % 6) / 10

# Merge batting and bowling stats
player_stats = pd.merge(
    batting_stats,
    bowling_stats,
    left_on=['match_id', 'batter'],
    right_on=['match_id', 'bowler'],
    how='outer'
)

# Combine player name
player_stats['player'] = player_stats['batter'].combine_first(player_stats['bowler'])

# Fill NaNs
player_stats = player_stats.fillna({
    'total_runs': 0,
    'balls_faced': 0,
    'strike_rate': 0,
    'dismissals': 0,
    'batting_avg': 0,
    'num_4s': 0,
    'num_6s': 0,
    'fifties': 0,
    'centuries': 0,
    'legal_deliveries': 0,
    'wickets': 0,
    'overs': 0
})

# Final columns
final_stats = player_stats[[
    'match_id', 'player', 'total_runs', 'balls_faced', 'num_4s', 'num_6s',
    'strike_rate', 'batting_avg', 'fifties', 'centuries',
    'legal_deliveries', 'overs', 'wickets'
]]

# Save to CSV
final_stats.to_csv("matchwise_detailed_player_performance.csv", index=False)

print("✅ Player match-wise performance stats saved to matchwise_detailed_player_performance.csv")


✅ Player match-wise performance stats saved to matchwise_detailed_player_performance.csv


In [20]:
import pandas as pd
import ast

# Load datasets
df1 = pd.read_csv("matchwise_detailed_player_performance.csv")
df2 = pd.read_csv("processed_match_data.csv")

# Safe evaluation function for player list columns
def safe_eval_players(cell):
    if pd.isna(cell):
        return []
    try:
        return ast.literal_eval(cell)[0]
    except Exception:
        return []

# Parse player lists
df2['team1_players'] = df2['team1_players'].apply(safe_eval_players)
df2['team2_players'] = df2['team2_players'].apply(safe_eval_players)

# Create match_id to player list mapping
match_team_players = df2.set_index('match_id')[['team1_players', 'team2_players']].to_dict('index')

# Function to determine player's team
def determine_team(row, team1_players, team2_players):
    player = row['player']
    if player in team1_players:
        return 'team1'
    elif player in team2_players:
        return 'team2'
    else:
        return 'unknown'

# Assign team information
df1['team'] = df1.apply(
    lambda row: determine_team(
        row,
        match_team_players.get(row['match_id'], {}).get('team1_players', []),
        match_team_players.get(row['match_id'], {}).get('team2_players', [])
    ), axis=1
)

# Merge datasets on match_id
merged_df = pd.merge(df1, df2, on='match_id', how='left')

# Save the result
merged_df.to_csv("merged_match_player_data.csv", index=False)
print("Merged dataset saved as 'merged_match_player_data.csv'")


Merged dataset saved as 'merged_match_player_data.csv'


In [26]:
import pandas as pd

# Load your data
match_info_df = pd.read_csv("Match_Info.csv")
merged_match_player_data_df = pd.read_csv("merged_match_player_data.csv")

# Convert date columns to datetime
match_info_df['match_date'] = pd.to_datetime(match_info_df['match_date'])
merged_match_player_data_df['date'] = pd.to_datetime(merged_match_player_data_df['date'])

# Filter 2024 season data
match_info_2024 = match_info_df[match_info_df['match_date'].dt.year == 2024]
merged_2024 = merged_match_player_data_df[merged_match_player_data_df['date'].dt.year == 2024].copy()

# Create a mapping from match_id to team1/2 players
match_player_mapping = match_info_2024.set_index('match_number')[['team1_players', 'team2_players']].to_dict('index')

# Fill missing player lists
def fill_teams_safe(row):
    if row['match_id'] in match_player_mapping:
        teams = match_player_mapping[row['match_id']]
        if not isinstance(row['team1_players'], list) or len(str(row['team1_players'])) < 5:
            row['team1_players'] = [p.strip() for p in teams['team1_players'].split(',')]
        if not isinstance(row['team2_players'], list) or len(str(row['team2_players'])) < 5:
            row['team2_players'] = [p.strip() for p in teams['team2_players'].split(',')]
    return row

merged_2024 = merged_2024.apply(fill_teams_safe, axis=1)

# Resolve 'unknown' team
def resolve_team(row):
    if row.get('team', 'unknown') == 'unknown':
        if row['player'] in row['team1_players']:
            return 'team1'
        elif row['player'] in row['team2_players']:
            return 'team2'
    return row.get('team', 'unknown')

merged_2024['team'] = merged_2024.apply(resolve_team, axis=1)

# (Optional) Save the cleaned 2024 data
merged_2024.to_csv("cleaned_merged_2024.csv", index=False)

# Preview a few results
print(merged_2024[['match_id', 'date', 'player', 'team']].head())


       match_id       date       player   team
21715   1422119 2024-03-22    AM Rahane  team2
21716   1422119 2024-03-22    AS Joseph  team1
21717   1422119 2024-03-22   Anuj Rawat  team1
21718   1422119 2024-03-22      C Green  team1
21719   1422119 2024-03-22  DJ Mitchell  team2


In [27]:
import pandas as pd

# Load the datasets
original_data = pd.read_csv("merged_match_player_data.csv")  # Data excluding 2024
cleaned_2024_data = pd.read_csv("cleaned_merged_2024.csv")   # Cleaned 2024 data

# Combine the datasets
merged_full = pd.concat([original_data, cleaned_2024_data], ignore_index=True)

# Optional: sort by date or match_id for consistency
merged_full.sort_values(by=["date", "match_id"], inplace=True)

# Save to a new CSV
merged_full.to_csv("final_merged_match_player_data.csv", index=False)

# Preview
print(merged_full[['match_id', 'date', 'player', 'team']].head())


      match_id        date        player   team
2993  419135.0  01-04-2010     A Symonds  team2
2994  419135.0  01-04-2010    AB Agarkar  team1
2995  419135.0  01-04-2010  AC Gilchrist  team2
2996  419135.0  01-04-2010    AD Mathews  team1
2997  419135.0  01-04-2010     CA Pujara  team1
